# Implement Bert

In [54]:
#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

import re

import nltk
#nltk.download() #input: punkt

from nltk import tokenize

import statistics #calculate mean and others

import datetime

In [49]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [50]:
abstracts = pd.read_pickle("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/smaller-final-dataset.pkl")

In [51]:
abstracts

,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,nchar,final_frqwds_removed,PROJECT_TERMS,PROJECT_TITLE,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,1402,"[multiprotein, y_secretase, proteolytically_cl...",Active Sites; Affect; Alzheimer's Disease; Am...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,...,BOSTON,MA,021156110,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,2553,"[kissl, gene, encode, peptide, kisspeptin, bin...",Affect; Animal Model; Axon; Behavior; Behavio...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,...,SEATTLE,WA,981959472,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,1414,"[biophysical, basis, thermodynamics_kinetic, m...",Agreement; Antibodies; base; Binding; Biochem...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,...,CAMBRIDGE,MA,021385319,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,1545,"[obesity, adverse_pregnancyoutcome, great, hea...",African; Analysis of Variance; Asians; Birth;...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,...,HOUSTON,TX,770305400,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN
4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,271,"[local, potato, advisory, express, interest, m...",cost; Health; interest; Manure; Parasitic nem...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,...,CORVALLIS,OR,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690809,1034758,1126666,Project Summary/AbstractThe pathophysiology of...,2019,1,1,2836,"[pathophysiology, schizophrenia, advance, thed...",Adolescence; Adult; Adult Children; Affect; A...,GUT DYSBIOSIS AND 5-HT2A DYSREGULATION IN A PR...,...,RICHMOND,VA,232980568,UNITED STATES,9/10/2019,9/9/2020,93.242,2019,37602.0,NaN
690810,1074034,1109788,PROJECT SUMMARY:Alzheimer’s Disease (AD) and A...,2019,1,1,2899,"[alzheimer, ad, amyotrophic_lateral_sclerosis_...",abeta deposition; Acetylgalactosamine; Affect...,TISSUE-SPECIFIC MODULATION OF APOLIPOPROTEIN E...,...,WORCESTER,MA,016550002,UNITED STATES,9/10/2019,9/9/2020,93.866,2019,32209.0,NaN
690811,1029450,1038442,"Project SummaryThe highest rates of mortality,...",2018,1,1,2186,"[highest, mortality, acute, care, encounter, r...",Acute; Address; Adherence; Adolescent; Adoles...,THE FEASIBILITY OF A MEDICAL STUDENT MENTORING...,...,NEW BRUNSWICK,NJ,089011914,UNITED STATES,9/10/2018,9/9/2019,93.840,2018,34994.0,NaN
690812,1074409,1154490,PROJECT SUMMARY: Treatment paradigms for kidne...,2019,1,1,3056,"[paradigm, kidney, largely, stagnant, decade, ...",3-Dimensional; Acute; Acute Renal Failure wit...,STRATEGIES IN RENAL NANOMEDICINE TO IMPACT TRE...,...,CHICAGO,IL,606113152,UNITED STATES,9/10/2019,9/9/2020,93.847,2019,50016.0,NaN


In [56]:
# Creates a list containing len_sentences lists, each of k items, all set to 0
#Matrix = [[0 for x in range(k)] for y in range(len_sentences)] 
#Matrix

In [57]:
abstracts_try = abstracts.iloc[0:100]
abstracts_try

,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,nchar,final_frqwds_removed,PROJECT_TERMS,PROJECT_TITLE,...,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,1402,"[multiprotein, y_secretase, proteolytically_cl...",Active Sites; Affect; Alzheimer's Disease; Am...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,...,BOSTON,MA,021156110,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,2553,"[kissl, gene, encode, peptide, kisspeptin, bin...",Affect; Animal Model; Axon; Behavior; Behavio...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,...,SEATTLE,WA,981959472,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,1414,"[biophysical, basis, thermodynamics_kinetic, m...",Agreement; Antibodies; base; Binding; Biochem...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,...,CAMBRIDGE,MA,021385319,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,1545,"[obesity, adverse_pregnancyoutcome, great, hea...",African; Analysis of Variance; Asians; Birth;...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,...,HOUSTON,TX,770305400,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN
4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,271,"[local, potato, advisory, express, interest, m...",cost; Health; interest; Manure; Parasitic nem...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,...,CORVALLIS,OR,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,758240,741964,DESCRIPTION (provided by applicant): Neuro-inf...,2015,1,1,1724,"[neuro_inflammation, immunosuppression, gliobl...",activating transcription factor; Affect; angi...,ROLE AND THERAPEUTIC VALUE OF AHR IN INFLAMMAT...,...,BOSTON,MA,021156110,UNITED STATES,5/1/2015,1/1/2016,93.853,2015,1.0,NaN
96,839161,915408,To determine the incidence and severity of sle...,2016,1,1,270,"[incidence, severity, sleep_apnea, distinguish...",cognitive function; Incidence; indexing; Medi...,MEDIATION EFFECTS OF SLEEP APNEA ON INDICES OF...,...,SILVER SPRING,MD,20910,UNITED STATES,NaN,NaN,NaN,2016,NaN,NaN
97,884503,861883,﻿ DESCRIPTION (provided by applicant): Oncol...,2016,1,1,3293,"[oncolytic_herpes_simplex_virus, hsv, promise,...",Adult; adult stem cell; Apoptosis; Apoptotic;...,IN VIVO IMAGING OF STEM CELL LOADED ONCOLYTIC ...,...,BOSTON,MA,021142696,UNITED STATES,3/1/2016,1/1/2017,93.395,2016,374428.0,NaN
98,857180,833135,﻿ DESCRIPTION (provided by applicant): The c...,2016,1,1,3129,"[career, k01, candidate, independent, preventi...",abeta accumulation; Affect; aging brain; Alle...,INFLUENCE OF GENETIC RISK FACTORS ON BIOMARKER...,...,BOSTON,MA,021142696,UNITED STATES,5/15/2016,1/1/2017,93.866,2016,131760.0,NaN


# Functions

In [58]:
def get_corpus_embeddings(dir):
    with open(dir) as f:
        ai_text = f.read()
    ai_corpus = tokenize.sent_tokenize(ai_text) #sentence tokenization
    ai_embeddings = embedder.encode(ai_corpus, show_progress_bar=True) # embeddings
    return ai_embeddings


# k: number of similar sentences from AI corpus
# abstract: abstract from FEDERAL RePORTER
# print_result: if TRUE, print out the similar sentenses from AI corpus to each sentence in the abstract
def get_score(k, abstract, print_result = False):
    queries = tokenize.sent_tokenize(abstract) 

    # init a result list for scores
    result = []
    
    # Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
    top_k = min(k, len(ai_embeddings))
    
    for query in queries: #compare each sentence in the abstract to the ai corpus
        query_embedding = embedder.encode(query, show_progress_bar=False) 
        
        # We use cosine-similarity and torch.topk to find the highest k scores
        cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
        
        top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
        result.append(top_results.values.tolist()) #unlist the top result list
        if print_result:
            print("\n\n======================\n\n")
            print("Query:", query)
            print("Results:", top_results)
            print("\nTop k=5 most similar sentences in corpus:")
            for score, idx in zip(top_results[0], top_results[1]):
                print(ai_corpus[idx], "(Score: {:.4f})".format(score))
    return result

In [60]:
start_time = datetime.datetime.now()
print("Start time:", start_time)

ai_embeddings = get_corpus_embeddings("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/ai_wiki_text.txt")

abstracts_try = abstracts_try.assign(score= abstracts_try["ABSTRACT"].apply(lambda x: get_score(10,x,False)))

sentence_score= []
for abstract in abstracts_try["score"]:
    sentence_score.append([statistics.mean(i) for i in abstract])

abstracts_try["sentence_score"]=sentence_score
abstracts_try

end_time = datetime.datetime.now()
print("Finished calculating ", len(abstracts_try), "of", "embedding score at", end_time)
print("It took", end_time-start_time, "to run.")

Start time: 2021-07-08 18:33:58.872627


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Finished calculating  100 of embedding score at 2021-07-08 18:34:08.535645
It took 0:00:09.663018 to run.


,PROJECT_TITLE,PROJECT_TERMS,score
0,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,Active Sites; Affect; Alzheimer's Disease; Am...,"[[0.30528321862220764, 0.2792899012565613, 0.2..."
1,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,Affect; Animal Model; Axon; Behavior; Behavio...,"[[0.22555223107337952, 0.2161628156900406, 0.2..."
2,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,Agreement; Antibodies; base; Binding; Biochem...,"[[0.4233601689338684, 0.4173330068588257, 0.41..."
3,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,African; Analysis of Variance; Asians; Birth;...,"[[0.2699534595012665, 0.26451870799064636, 0.2..."
4,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,cost; Health; interest; Manure; Parasitic nem...,"[[0.39741647243499756, 0.3675093650817871, 0.3..."
5,WNT SIGNALING IN THE INDUCTION OF NEW TOOTH FO...,Adenovirus Vector; Adenoviruses; Adult; base;...,"[[0.36166852712631226, 0.2949758768081665, 0.2..."
6,CHARACTERIZATION AND CARE COORDINATION OF POLY...,Caring; Characteristics; Clinical; follow-up;...,"[[0.3051285147666931, 0.301409512758255, 0.290..."
7,PEDIATRIC PHARMACOLOGY RESEARCH UNITS COORD CTR,NaN,"[[0.2824210524559021, 0.2756810486316681, 0.25..."
8,SYNAPTIC AND INTRINSIC EXITABILITY IN MOTONEUR...,Acute; Adolescent; Adult; Age; Alanine; Back;...,"[[0.4059833288192749, 0.33709877729415894, 0.3..."
9,CLOUD PROPERTIES FROM REFLECTED LIGHT. EPOCH''...,Light; Measurement; planetary Atmosphere; Pla...,"[[0.26459598541259766, 0.2325885146856308, 0.2..."


[[0.30528321862220764,
  0.2792899012565613,
  0.2571047842502594,
  0.25513893365859985,
  0.24579471349716187,
  0.23625369369983673,
  0.23303790390491486,
  0.2250896841287613,
  0.22023862600326538,
  0.2173820286989212],
 [0.283286988735199,
  0.24036338925361633,
  0.22596217691898346,
  0.2156960368156433,
  0.21008829772472382,
  0.20962612330913544,
  0.20712365210056305,
  0.20476365089416504,
  0.1945364624261856,
  0.19306619465351105],
 [0.23714688420295715,
  0.2288617342710495,
  0.19803330302238464,
  0.19169002771377563,
  0.19008196890354156,
  0.18523816764354706,
  0.1837732046842575,
  0.1816103756427765,
  0.17859548330307007,
  0.16819077730178833],
 [0.24819616973400116,
  0.24185621738433838,
  0.24015043675899506,
  0.23495443165302277,
  0.22512878477573395,
  0.22220392525196075,
  0.21714305877685547,
  0.21489596366882324,
  0.21268518269062042,
  0.21152856945991516],
 [0.35369348526000977,
  0.3480268120765686,
  0.34085220098495483,
  0.335095882415771

,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,nchar,final_frqwds_removed,PROJECT_TERMS,PROJECT_TITLE,...,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,score,sentence_score
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,1402,"[multiprotein, y_secretase, proteolytically_cl...",Active Sites; Affect; Alzheimer's Disease; Am...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,...,021156110,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN,"[[0.30528321862220764, 0.2792899012565613, 0.2...","[0.24746134877204895, 0.2184512972831726, 0.19..."
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,2553,"[kissl, gene, encode, peptide, kisspeptin, bin...",Affect; Animal Model; Axon; Behavior; Behavio...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,...,981959472,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN,"[[0.22555223107337952, 0.2161628156900406, 0.2...","[0.19784948378801345, 0.18860326260328292, 0.2..."
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,1414,"[biophysical, basis, thermodynamics_kinetic, m...",Agreement; Antibodies; base; Binding; Biochem...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,...,021385319,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN,"[[0.4233601689338684, 0.4173330068588257, 0.41...","[0.38222160637378694, 0.3518401920795441, 0.29..."
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,1545,"[obesity, adverse_pregnancyoutcome, great, hea...",African; Analysis of Variance; Asians; Birth;...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,...,770305400,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN,"[[0.2699534595012665, 0.26451870799064636, 0.2...","[0.2295188069343567, 0.22275962084531784, 0.20..."
4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,271,"[local, potato, advisory, express, interest, m...",cost; Health; interest; Manure; Parasitic nem...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,...,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN,"[[0.39741647243499756, 0.3675093650817871, 0.3...","[0.31080458760261537, 0.2393555074930191, 0.23..."
5,233317,264035,DESCRIPTION (provided by applicant): Despite a...,2009,1,1,2573,"[advance, tooth, morphogenesis, differentiatio...",Adenovirus Vector; Adenoviruses; Adult; base;...,WNT SIGNALING IN THE INDUCTION OF NEW TOOTH FO...,...,021156110,UNITED STATES,7/1/2009,1/1/2010,93.121,2009,90000.0,NaN,"[[0.36166852712631226, 0.2949758768081665, 0.2...","[0.27807527780532837, 0.20416890680789948, 0.1..."
6,181828,915146,To describe the short-term durability of funct...,2008,1,1,487,"[describe, short, durability, functional, vete...",Caring; Characteristics; Clinical; follow-up;...,CHARACTERIZATION AND CARE COORDINATION OF POLY...,...,20910,UNITED STATES,NaN,NaN,NaN,2008,NaN,NaN,"[[0.3051285147666931, 0.301409512758255, 0.290...","[0.2724839597940445, 0.3702690452337265, 0.181..."
7,286624,692386,The NICHD's Pediatric Pharmacology Research Un...,2009,1,1,1290,"[nichd, pediatric, pharmacology, units, ppru, ...",NaN,PEDIATRIC PHARMACOLOGY RESEARCH UNITS COORD CTR,...,208523093,UNITED STATES,NaN,NaN,NaN,2009,969123.0,NaN,"[[0.2824210524559021, 0.2756810486316681, 0.25...","[0.2443540632724762, 0.23413871973752975, 0.25..."
8,212518,222762,DESCRIPTION (provided by applicant): This stud...,2009,1,1,2738,"[postnatal, intrinsic, synaptic, excitability,...",Acute; Adolescent; Adult; Age; Alanine; Back;...,SYNAPTIC AND INTRINSIC EXITABILITY IN MOTONEUR...,...,606113152,UNITED STATES,1/2/2009,1/1/2010,93.853,2009,55310.0,NaN,"[[0.4059833288192749, 0.33709877729415894, 0.3...","[0.3219610095024109, 0.25245423018932345, 0.26..."
9,13107,102277,Cloud properties from reflected light. EPOCh''...,2008,1,1,255,"[cloud, property, reflect, light, epoch, refle...",Light; Measurement; planetary Atmosphere; Pla...,CLOUD PROPERTIES FROM REFLEC

In [46]:
abstracts_try = abstracts_try.assign(abstract_score= abstracts_try["sentence_score"].apply(lambda x: statistics.mean(x)))

In [47]:
abstracts_try

,original index,PROJECT_ID,ABSTRACT,FY,ORG_COUNT,PI_COUNT,nchar,final_frqwds_removed,PROJECT_TERMS,PROJECT_TITLE,...,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,score,sentence_score,abstract_score
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,1,1,1402,"[multiprotein, y_secretase, proteolytically_cl...",Active Sites; Affect; Alzheimer's Disease; Am...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,...,UNITED STATES,12/1/2007,1/1/2008,93.866,2008,3483.0,NaN,"[[0.30528321862220764, 0.2792899012565613, 0.2...","[0.24746134877204895, 0.2184512972831726, 0.19...",0.263356
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,1,1,2553,"[kissl, gene, encode, peptide, kisspeptin, bin...",Affect; Animal Model; Axon; Behavior; Behavio...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,...,UNITED STATES,9/1/2008,1/1/2009,93.865,2008,39175.0,NaN,"[[0.22555223107337952, 0.2161628156900406, 0.2...","[0.19784948378801345, 0.18860326260328292, 0.2...",0.246339
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,1,1,1414,"[biophysical, basis, thermodynamics_kinetic, m...",Agreement; Antibodies; base; Binding; Biochem...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,...,UNITED STATES,1/2/2008,1/1/2009,93.859,2008,49646.0,NaN,"[[0.4233601689338684, 0.4173330068588257, 0.41...","[0.38222160637378694, 0.3518401920795441, 0.29...",0.338063
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,1,1,1545,"[obesity, adverse_pregnancyoutcome, great, hea...",African; Analysis of Variance; Asians; Birth;...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,...,UNITED STATES,4/1/2008,1/1/2009,93.361,2008,20406.0,NaN,"[[0.2699534595012665, 0.26451870799064636, 0.2...","[0.2295188069343567, 0.22275962084531784, 0.20...",0.262953
4,371628,594482,Local potato advisory groups have expressed in...,2010,1,1,271,"[local, potato, advisory, express, interest, m...",cost; Health; interest; Manure; Parasitic nem...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,...,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN,"[[0.39741647243499756, 0.3675093650817871, 0.3...","[0.31080458760261537, 0.2393555074930191, 0.23...",0.263024
5,233317,264035,DESCRIPTION (provided by applicant): Despite a...,2009,1,1,2573,"[advance, tooth, morphogenesis, differentiatio...",Adenovirus Vector; Adenoviruses; Adult; base;...,WNT SIGNALING IN THE INDUCTION OF NEW TOOTH FO...,...,UNITED STATES,7/1/2009,1/1/2010,93.121,2009,90000.0,NaN,"[[0.36166852712631226, 0.2949758768081665, 0.2...","[0.27807527780532837, 0.20416890680789948, 0.1...",0.282890
6,181828,915146,To describe the short-term durability of funct...,2008,1,1,487,"[describe, short, durability, functional, vete...",Caring; Characteristics; Clinical; follow-up;...,CHARACTERIZATION AND CARE COORDINATION OF POLY...,...,UNITED STATES,NaN,NaN,NaN,2008,NaN,NaN,"[[0.3051285147666931, 0.301409512758255, 0.290...","[0.2724839597940445, 0.3702690452337265, 0.181...",0.258419
7,286624,692386,The NICHD's Pediatric Pharmacology Research Un...,2009,1,1,1290,"[nichd, pediatric, pharmacology, units, ppru, ...",NaN,PEDIATRIC PHARMACOLOGY RESEARCH UNITS COORD CTR,...,UNITED STATES,NaN,NaN,NaN,2009,969123.0,NaN,"[[0.2824210524559021, 0.2756810486316681, 0.25...","[0.2443540632724762, 0.23413871973752975, 0.25...",0.268511
8,212518,222762,DESCRIPTION (provided by applicant): This stud...,2009,1,1,2738,"[postnatal, intrinsic, synaptic, excitability,...",Acute; Adolescent; Adult; Age; Alanine; Back;...,SYNAPTIC AND INTRINSIC EXITABILITY IN MOTONEUR...,...,UNITED STATES,1/2/2009,1/1/2010,93.853,2009,55310.0,NaN,"[[0.4059833288192749, 0.33709877729415894, 0.3...","[0.3219610095024109, 0.25245423018932345, 0.26...",0.268396
9,13107,102277,Cloud properties from reflected light. EPOCh''...,2008,1,1,255,"[cloud, property, reflect, light, epoch, refle...",Light; Measurement; planetary Atmosphere; Pla...,CLOUD PROPERTIES FROM REFLECTED